In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
vote_file = pd.read_csv('brexit_full_updated3.csv')
vote_file

,Constituency ID,full name,PANO,Constituency Name,Party abbreviation,conShare,conShare2,conShare3,bill161,leave,party_old,ref,ref_dummy,party,old_party
0,E14000530,Gerald Howarth,7,Aldershot,Con,50.59,73.403947,73.403947,1.0,1.0,1.0,0.578978,1,1.0,1.0
1,E14000531,Wendy Morton,8,Aldridge-Brownhills,Con,52.05,69.940876,69.940876,1.0,0.0,1.0,0.677963,1,0.0,0.0
2,E14000532,Graham Brady,9,Altrincham and Sale West,Con,52.99,66.503514,66.503514,1.0,1.0,1.0,0.385878,0,1.0,1.0
3,E14000533,Nigel Mills,11,Amber Valley,Con,43.98,55.840528,55.840528,1.0,1.0,1.0,0.652991,1,1.0,1.0
4,E14000534,Nick Herbert,18,Arundel and South Downs,Con,60.79,84.442284,84.442284,NaN,0.0,1.0,0.497011,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,W07000076,Wayne David,114,Caerphilly,Lab,16.59,27.223499,27.223499,1.0,0.0,0.0,0.551360,1,0.0,NaN
656,W07000077,Chris Evans,336,Islwyn,Lab,15.16,23.639482,23.639482,1.0,0.0,0.0,0.589399,1,0.0,NaN
657,W07000078,Alun Cairns,589,Vale Of Glamorgan,Con,46.02,58.527280,58.527280,1.0,0.0,1.0,0.525507,1,0.0,0.0
658,W07000079,Kevin Brennan,129,Cardiff West,Lab,25.15,38.221884,38.221884,0.0,0.0,0.0,0.438226,0,0.0,NaN


In [4]:
politicians_party = {}
politicians = vote_file['full name'].unique()
for index, row in vote_file.iterrows():
    politicians_party[row['full name']] = row['party']

In [210]:
df1 = pd.read_csv('2015_commons.csv')
df2 = pd.read_csv('2016_commons.csv')
df = pd.concat([df1, df2])
df = df.drop(['Government'], axis=1)
df = df.loc[(df['Party'] == 'Conservative') | (df['Party'] == 'Labour')]
df = df.loc[df['Name'].isin(politicians)]
df['Date'] = pd.to_datetime(df['Date'])
mask = (df['Date'] > '2015-5-1') & (df['Date'] <= '2016-6-30')
df = df.loc[mask]
df['party2'] = df['Name'].apply(lambda x: politicians_party[x])
# df = df.loc[df['Topic'] == 'European Union']
cols = df.columns
cols = ['date', 'speaker', 'name', 'Party', 'Topic', 'transcript', 'party2']
df.columns = cols

In [211]:
# df = df.loc[df['Topic'] == 'European Union']

In [212]:
df.head()

,date,speaker,name,Party,Topic,transcript,party2
18469,2015-05-27,Mr Simon Burns (Chelmsford) (Con),Simon Burns,Conservative,admin,"I beg to move, That an humble Address be pres...",0.0
18470,2015-05-27,Mrs Sheryll Murray (South East Cornwall) (Con),Sheryll Murray,Conservative,admin,It is an honour to be invited to second this H...,1.0
18471,2015-05-27,Ms Harriet Harman (Camberwell and Peckham) (Lab),Harriet Harman,Labour,admin,I am sure the whole House will want to pay tri...,0.0
18472,2015-05-27,The Prime Minister (Mr David Cameron),David Cameron,Conservative,admin,"As we meet today, we should start, as the righ...",0.0
18473,2015-05-27,Mr David Winnick (Walsall North) (Lab),David Winnick,Labour,admin,The right hon. Gentleman is talking about “one...,0.0


In [213]:
leave_conservative = df.loc[(df['Party'] == 'Conservative') & (df['party2'] == 1)]
remain_labour = df.loc[(df['Party'] == 'Labour') & (df['party2'] == 0)]
remain_conservative = df.loc[(df['Party'] == 'Conservative') & (df['party2'] == 0)]

In [214]:
leave_conservative = ' '.join(leave_conservative['transcript'].values)
remain_labour = ' '.join(remain_labour['transcript'].values)
remain_conservative = ' '.join(remain_conservative['transcript'].values)

data = [leave_conservative, remain_labour, remain_conservative]


In [215]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(3,3))
tfidf = vectorizer.fit_transform(data)

In [216]:
vectorizer.get_feature_names()

['00 19 00',
 '00 capacity leigh',
 '00 cent gdp',
 '00 council tax',
 '00 hour 2020',
 '00 people 80s',
 '00 railway carpet',
 '00 tuesday december',
 '00 unacceptable tiny',
 '000 000 000',
 '000 000 2008',
 '000 000 2015',
 '000 000 2020',
 '000 000 announcement',
 '000 000 apprenticeship',
 '000 000 april',
 '000 000 bank',
 '000 000 begin',
 '000 000 certainly',
 '000 000 children',
 '000 000 constituents',
 '000 000 cutting',
 '000 000 employees',
 '000 000 equity',
 '000 000 extra',
 '000 000 extremely',
 '000 000 family',
 '000 000 help',
 '000 000 importantly',
 '000 000 inconsiderable',
 '000 000 insurance',
 '000 000 level',
 '000 000 making',
 '000 000 male',
 '000 000 means',
 '000 000 need',
 '000 000 officers',
 '000 000 particularly',
 '000 000 people',
 '000 000 petition',
 '000 000 pharmacies',
 '000 000 police',
 '000 000 policemen',
 '000 000 pro',
 '000 000 provide',
 '000 000 pupil',
 '000 000 rare',
 '000 000 rates',
 '000 000 recommendations',
 '000 000 right',


In [147]:
cosine_similarities = linear_kernel(tfidf, tfidf)

In [148]:
cosine_similarities

array([[1.        , 0.11908158, 0.20371289],
       [0.11908158, 1.        , 0.19264837],
       [0.20371289, 0.19264837, 1.        ]])

In [149]:
cols = ['type', 'leave_conservative', 'remain_labour', 'remain_conservative']

rows = []

for i in range(len(cosine_similarities)):
    row = [cols[i+1]] + list(cosine_similarities[i])
    rows.append(row)

res_df = pd.DataFrame(rows, columns=cols)
res_df

,type,leave_conservative,remain_labour,remain_conservative
0,leave_conservative,1.000000,0.119082,0.203713
1,remain_labour,0.119082,1.000000,0.192648
2,remain_conservative,0.203713,0.192648,1.000000


In [151]:
res_df.to_csv('./trigrams/similarity_health_medicine.csv')